In [1]:
import sys
sys.path.append("..")
import torch
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid", color_codes=True)
import train
import train_model_no_WS
from helpers.train_helpers import get_dataset, get_device, accuracy

%matplotlib inline

In [2]:
%load_ext tensorboard
%tensorboard --logdir ./logs

config_model_no_WS = dict(
    logs_dir=f'./logs/no_ws/{datetime.now().strftime("%Y%m%d_%H%M%S")}',
    optimizer='Adam',
    optimizer_learning_rate=0.001,
    batch_size=128,
    num_epochs=80,  # 250
    model='simple_conv',
    class_num=10,
    channels_in=1,
    augmentation=False,
    verbose=0
)

model = train_model_no_WS.train(config_model_no_WS)

Reusing TensorBoard on port 6006 (pid 27358), started 4:48:48 ago. (Use '!kill 27358' to kill it.)

In [3]:
config = dict(batch_size=1, augmentation=False)
training_loader, test_loader = get_dataset(config)
device = get_device()
criterion = torch.nn.CrossEntropyLoss()

In [14]:
for batch_x, batch_y, batch_classes in test_loader:

    batch_x_1 = batch_x[:, 0, :, :].view(-1, 1, batch_x.size(2), batch_x.size(3)).to(device)
    batch_x_2 = batch_x[:, 1, :, :].view(-1, 1, batch_x.size(2), batch_x.size(3)).to(device)

    batch_classes_1 = batch_classes[:, 0].to(device)
    batch_classes_2 = batch_classes[:, 1].to(device)
    print(batch_classes_1)
    print(batch_classes_2)

    print(batch_y)
    batch_y = batch_y.to(device)

    prediction_1 = model(batch_x_1)
    prediction_2 = model(batch_x_2)
    print(prediction_1)
    print(prediction_2)
    loss = (criterion(prediction_1, batch_classes_1) + criterion(prediction_2, batch_classes_2)) / 2
    print("loss=", loss)
    acc = accuracy(prediction_1.argmax(1) <= prediction_2.argmax(1), batch_y, argmax=False)
    print("acc=", acc)
    break

tensor([1], device='cuda:0')
tensor([9], device='cuda:0')
tensor([1])
tensor([[ -0.5814,  14.6352,  -9.5046,  -5.3171,  -0.8414,  -1.4323,  -2.9815,
         -11.1554,   7.7038,  -0.9001]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[ -3.5137, -26.8082, -14.8773,   2.0239,  10.7815,  -2.5424, -10.6222,
          -1.1454,   6.5301,  26.8084]], device='cuda:0',
       grad_fn=<AddmmBackward>)
loss= tensor(0.0005, device='cuda:0', grad_fn=<DivBackward0>)
acc= tensor(1., device='cuda:0')
